In [1]:
from src.preprocessing.xml_to_mask import process_slide

# Define paths for testing
wsi_path = "data/Yale_trastuzumab_response_cohort/SVS/O09-03495.svs"
xml_path = "data/Yale_trastuzumab_response_cohort/Annotations/O09-03495.xml"
output_path = "data/Yale_trastuzumab_response_cohort/Masks/O09-03495_mask.png"


# Test the process_slide function
process_slide(xml_path, wsi_path, output_path)

c:\Users\tanth\anaconda3\envs\her2-class\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Processing: data/Yale_trastuzumab_response_cohort/SVS/O09-03495.svs + data/Yale_trastuzumab_response_cohort/Annotations/O09-03495.xml
Slide dimensions: width=55216, height=30663
WSI image shape: (30663, 55216, 3)
WSI with mask saved to data/Yale_trastuzumab_response_cohort/Masks/O09-03495_mask_wsi_with_mask.png (tiled gpu)


'data/Yale_trastuzumab_response_cohort/Masks/O09-03495_mask_wsi_with_mask.png'